[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/casangi/sirius/blob/main/docs/technical_memo_notebooks/03_parallactic_angle.ipynb)

The purpose of this notebook is to compare the different ways parallactic angles are calculated using Astropy and CASA. The functions can be found in ```sirius/sirius/_sirius_utils/_calc_parallactic_angles.py```. 

- ```_calc_parallactic_angles_astropy```  Converts a direction and zenith (frame FK5) to a topocentric WGS84 Altitude-Azimuth (https://docs.astropy.org/en/stable/api/astropy.coordinates.AltAz.html) frame centered at the observing_location (frame ITRF) for a UTC time. The parallactic angles is calculated as the position angle of the Altitude-Azimuth direction and zenith.
- ```_calc_parallactic_angles_astropy2``` Uses the Astroplan Observer container class.
- ```_calc_parallactic_angles_casa``` Uses the same approach as ```_calc_parallactic_angles_astropy``` except using the Casacore measures tool and the earth model can be specified (frame='AZEL' (spherical) or AZELGEO (WGS84)).
- ```_calc_parallactic_angles_casa2``` Uses the approach in https://github.com/ARDG-NRAO/plumber/blob/master/plumber/sky.py.
- ```_calc_parallactic_angles_casa3``` Based on ```casacore/ms/MSOper/MSDerivedValues.cc::parAngle()``` and ```casacore/casa/Quanta/MVDirection.cc::positionAngle```.

All of the functions will be compared to ```_calc_parallactic_angles_casa```, since this function allows for the earth model can be specified. The parallactic angles calculated by the awproject gridder will also be compared.

## Astropy
![title](astropy_pa.png)

## CASA
![title](casa_pa.png)

In [1]:
import os
try:
    import sirius 
    print('SiRIUS version',sirius.__version__,'already installed.')
except ImportError as e:
    print(e)
    print('Installing SiRIUS')
    os.system("pip install sirius")
    import sirius 
    print('SiRIUS version',sirius.__version__,' installed.')



SiRIUS version 0.0.28 already installed.


## Load Packages

In [2]:
import pkg_resources
import xarray as xr
import numpy as np
from astropy.coordinates import SkyCoord
from datetime import datetime

xr.set_options(display_style="html")
import os
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    IN_COLAB = True
except:
    IN_COLAB = False
    
#Create directory to store generated data
data_dir = 'data/'
os.makedirs(data_dir, exist_ok=True)

#Move casa log to data dir
from casatasks import casalog
now = datetime.now()
dt_string = now.strftime("-%d%m%Y-%H%M%S")
casalog.setlogfile(data_dir+'casa'+dt_string+'.log')
    
#interactive plots
#%matplotlib widget 
#embeded plots 
%matplotlib inline 

To calculate a parallatic angle we need the following info:

- Date time 
- Earth location
- Direction

For the VLA the antenna pads have been tilted in such a way so that a single earth location can be used to calculate the parallatic angle for all antennas. This is however not the case for ALMA. The awproject gridder in tclean uses the position of the first antenna to calculate the parallactic angle for the array.

The array center which is used for the correlator point (origin for UVW) and the position on Earth where the TOPO frequency is measured for that telescope can be obtained using:

In [3]:
from casacore.measures import measures
me = measures()
site_pos =  me.observatory('EVLA')
site_pos=me.measure(me.observatory('EVLA'),'ITRF')
print(site_pos)
site_pos=me.measure(me.observatory('EVLA'),'WGS84')
print(site_pos)

{'type': 'position', 'refer': 'ITRF', 'm2': {'value': 6373580.000000001, 'unit': 'm'}, 'm1': {'unit': 'rad', 'value': 0.5916753430723376}, 'm0': {'unit': 'rad', 'value': -1.8782884344112576}}
{'type': 'position', 'refer': 'WGS84', 'm2': {'value': 2118.614142014645, 'unit': 'm'}, 'm1': {'unit': 'rad', 'value': 0.5947877410277558}, 'm0': {'unit': 'rad', 'value': -1.8782884344112576}}


- ITRF is a geocentric 
- WGS84 is a geodetic

## CASA Sim using Awprojewct

In [4]:
if not os.path.isdir(data_dir+'sim_awproject.ms'):
    from scripts.awproject_sim_03 import sim_awproject_evla
    sim_awproject_evla(data_dir+'sim_awproject.ms')

## SiRIUS Sim

In [5]:
if not os.path.isdir(data_dir+'sim_sirius.ms'):
    from scripts.sirius_sim_03 import sim_sirius_evla
    sim_sirius_evla(data_dir+'sim_sirius.ms')

## Awproject Parallactic Angles

Extract parallactic angles from cfcache.

In [6]:
from casatools import image
ia = image()
awproject_pa = np.zeros(6)

for i in range(6):
    ia.open(data_dir+'img_sim_awproject.cf/'+'CFS_'+str(i)+'_0_CF_0_0_0.im')
    miscinfo = ia.miscinfo()
    awproject_pa[i] = miscinfo['ParallacticAngle']
    ia.close()
    
print("Parallactic angles in degrees",awproject_pa)



Parallactic angles in degrees [ -70.649899    -86.29112099 -115.71606771  114.72711096   85.99807173
   70.39997555]


## SiRIUS Parallactic Angles

In [13]:
def fix_ang(a):
    a[a > np.pi]= a[a > np.pi] - 2*np.pi
    a = a*180/np.pi
    return a

from sirius._sirius_utils._calc_parallactic_angles import _calc_parallactic_angles_astropy, _calc_parallactic_angles_astropy2, _calc_parallactic_angles_casa, _calc_parallactic_angles_casa2, _calc_parallactic_angles_casa3
from casatools import msmetadata, measures, quanta
from astropy.time import Time
import numpy as np
import pkg_resources
import xarray as xr
msmd = msmetadata()
me = measures()

# Get Time Steps
field = 'field1'
#msmd.open('data/sim_awproject.ms')
msmd.open('data/sim_sirius.ms')
times = msmd.timesforfield(msmd.fieldsforname(field)[0])
time_str = Time(times/(3600.* 24), format='mjd').isot

'''
# Array Center
from sirius._sirius_utils._coord_transforms import _convert_latlong_to_xyz
site_pos =  me.observatory('EVLA')
site_pos=me.measure(me.observatory('EVLA'),'ITRF')
_convert_latlong_to_xyz(site_pos)
observing_location = np.array([site_pos['m0']['value'],site_pos['m1']['value'],site_pos['m2']['value']])
'''

# awproject uses the position of the first antenna to calculate the parallactic angle
tel_dir = pkg_resources.resource_filename('sirius_data', 'telescope_layout/data/evla.d.tel.zarr')
tel_xds = xr.open_zarr(tel_dir,consolidated=False)
observing_location = tel_xds.ANT_POS[0,:].values

phase_center = msmd.phasecenter(msmd.fieldsforname(field)[0])
phase_center_ra_dec = np.array([phase_center['m0']['value'],phase_center['m1']['value']])[None,:]


# What does awproject in CASA do
pa_ang_sirius_casa = fix_ang(_calc_parallactic_angles_casa(time_str, observing_location, phase_center_ra_dec, frame='AZEL', dir_frame='FK5', zenith_frame='HADEC'))
print('awproject parallactic angle degrees ',awproject_pa)
print('casa parallactic angle degrees',pa_ang_sirius_casa)
print('Difference in arcseconds ',(awproject_pa-pa_ang_sirius_casa)*(3600))
print('****************')

# Replicate other astropy and casa functions.

pa_ang_sirius_astropy = fix_ang(_calc_parallactic_angles_astropy(time_str, observing_location, phase_center_ra_dec, dir_frame='FK5'))
pa_ang_sirius_casa = fix_ang(_calc_parallactic_angles_casa(time_str, observing_location, phase_center_ra_dec, frame='AZELGEO', dir_frame='FK5', zenith_frame='FK5'))
print('astropy parallactic angle degrees',pa_ang_sirius_astropy)
print('casa parallactic angle degrees',pa_ang_sirius_casa)
print('Difference in arcseconds ' ,(pa_ang_sirius_astropy-pa_ang_sirius_casa)*3600)
print('****************')

pa_ang_sirius_astropy = fix_ang(_calc_parallactic_angles_astropy(time_str, observing_location, phase_center_ra_dec, dir_frame='FK5'))
pa_ang_sirius_casa = fix_ang(_calc_parallactic_angles_casa(time_str, observing_location, phase_center_ra_dec, frame='AZEL', dir_frame='FK5', zenith_frame='HADEC'))
print('astropy parallactic angle degrees',pa_ang_sirius_astropy)
print('casa parallactic angle degrees',pa_ang_sirius_casa)
print('Difference in arcseconds ' ,(pa_ang_sirius_astropy-pa_ang_sirius_casa)*3600)
print('****************')

pa_ang_sirius_astropy2 = fix_ang(_calc_parallactic_angles_astropy2(time_str, observing_location, phase_center_ra_dec, dir_frame='FK5'))
pa_ang_sirius_casa = fix_ang(_calc_parallactic_angles_casa(time_str, observing_location, phase_center_ra_dec, frame='AZEL', dir_frame='FK5', zenith_frame='FK5'))
print('astropy2 parallactic angle degrees',pa_ang_sirius_astropy2)
print('casa parallactic angle degrees',pa_ang_sirius_casa)
print('Difference in arcseconds ' ,(pa_ang_sirius_astropy2-pa_ang_sirius_casa)*3600)
print('****************')

pa_ang_sirius_casa2 = fix_ang(_calc_parallactic_angles_casa2(time_str, observing_location, phase_center_ra_dec, dir_frame='FK5'))
pa_ang_sirius_casa = fix_ang(_calc_parallactic_angles_casa(time_str, observing_location, phase_center_ra_dec, frame='AZEL', dir_frame='FK5', zenith_frame='HADEC'))
print('casa2 parallactic angle degrees',pa_ang_sirius_casa2)
print('casa parallactic angle degrees',pa_ang_sirius_casa)
print('Difference in arcseconds ' ,(pa_ang_sirius_casa2-pa_ang_sirius_casa2)*3600)
print('****************')

pa_ang_sirius_casa3 = fix_ang(_calc_parallactic_angles_casa3(time_str, observing_location, phase_center_ra_dec, frame='AZEL', dir_frame='FK5', zenith_frame='HADEC'))
pa_ang_sirius_casa = fix_ang(_calc_parallactic_angles_casa(time_str, observing_location, phase_center_ra_dec, frame='AZEL', dir_frame='FK5', zenith_frame='HADEC'))
print('casa3 parallactic angle degrees',pa_ang_sirius_casa)
print('casa parallactic angle degrees',pa_ang_sirius_casa)
print((pa_ang_sirius_casa-pa_ang_sirius_casa3)*3600)
print('****************')

#Zenith starting frame 'HADEC' vs 'FK5'
pa_ang_sirius_casa_HADEC = fix_ang(_calc_parallactic_angles_casa(time_str, observing_location, phase_center_ra_dec, frame='AZELGEO', dir_frame='FK5', zenith_frame='HADEC'))
pa_ang_sirius_casa_FK5 = fix_ang(_calc_parallactic_angles_casa(time_str, observing_location, phase_center_ra_dec, frame='AZELGEO', dir_frame='FK5', zenith_frame='FK5'))
print('casa_HADEC parallactic angle degrees',pa_ang_sirius_casa_HADEC)
print('casa_FK5 parallactic angle degrees',pa_ang_sirius_casa_FK5)
print('Difference in arcseconds ' ,(pa_ang_sirius_casa_HADEC-pa_ang_sirius_casa_FK5)*3600)
print('****************')

#Frame to convert to 'AZELGEO' vs 'AZEL'
pa_ang_sirius_casa_AZELGEO = fix_ang(_calc_parallactic_angles_casa(time_str, observing_location, phase_center_ra_dec, frame='AZELGEO', dir_frame='FK5', zenith_frame='FK5'))
pa_ang_sirius_casa_AZEL = fix_ang(_calc_parallactic_angles_casa(time_str, observing_location, phase_center_ra_dec, frame='AZEL', dir_frame='FK5', zenith_frame='FK5'))
print('casa_AZELGEO parallactic angle degrees',pa_ang_sirius_casa_AZELGEO)
print('casa_AZEL parallactic angle degrees',pa_ang_sirius_casa_AZEL)
print('Difference in arcseconds ' ,(pa_ang_sirius_casa_AZELGEO-pa_ang_sirius_casa_AZEL)*3600)

#Questions FK5 (J2000) -> ICRS
#For PA calc AZEL vs AZELGEO
#Zenith for PA calc FK5 or HADEC

awproject parallactic angle degrees  [ -70.649899    -86.29112099 -115.71606771  114.72711096   85.99807173
   70.39997555]
casa parallactic angle degrees [ -70.64989838  -86.29112085 -115.7160621   114.72711189   85.99807229
   70.39997512]
Difference in arcseconds  [-0.00222751 -0.00052524 -0.0202098  -0.00334638 -0.00203884  0.0015493 ]
****************
astropy parallactic angle degrees [ -70.34777117  -85.89276089 -114.96699024  114.23132453   85.84484406
   70.341338  ]
casa parallactic angle degrees [ -70.34778111  -85.89275009 -114.96710083  114.2309824    85.84475879
   70.34128818]
Difference in arcseconds  [ 0.03577029 -0.03888978  0.39811484  1.23167225  0.30695619  0.17934905]
****************
astropy parallactic angle degrees [ -70.34777117  -85.89276089 -114.96699024  114.23132453   85.84484406
   70.341338  ]
casa parallactic angle degrees [ -70.64989838  -86.29112085 -115.7160621   114.72711189   85.99807229
   70.39997512]
Difference in arcseconds  [ 1087.65796601  143

- (PA_AZELGEO \- PA_AZEL) is a function of elevation. The closer to the horizon, the smaller the error.
- (PA_HADEC \- PA_FK5) is almost constant.